In [1]:
import pandas as pd
import numpy as np
import re

from functools import reduce

import general_rulings_map

pd.set_option('display.max_rows', 500)

In [2]:
master = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Master')
bonus = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Bonus cards')

/home/navarog/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
# Isolate later expansions
oceania = master[master['Expansion'] == 'oceania']
oceania = oceania.sort_values(by=['Common name'], na_position='first')

asia = master[master['Expansion'] == 'asia']
asia = asia.sort_values(by=['Common name'], na_position='first')

# Index according to legacy rules 
master = master[(master['Expansion'].isin(['originalcore', 'core', 'swiftstart', 'european', 'chinesepromo']) | pd.isna(master['Expansion']))]
master = master.sort_values(by=['Common name'], na_position='first')

### Append later expansions
master = master.append(oceania)
master = master.append(asia)

master = master.reset_index(drop=True)
master = master.drop([0, 1])

In [4]:
master['id'] = master.index
master['Common name'] = master['Common name'].map(lambda s: s.strip())
bonus['id'] = bonus.index + 1000

In [5]:
master = master[master['Expansion'] != 'chinesepromo']
master['Nest type'].loc[pd.isna(master['Nest type'])] = 'None'

/home/navarog/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
master_ids = reduce(lambda acc, val: {**acc, val[1]['Common name']: val[1]['id']}, master.iterrows(), {})
bonus_ids = reduce(lambda acc, val: {**acc, val[1]['Name']: val[1]['id']}, bonus.iterrows(), {})
ids = {**master_ids, **bonus_ids}

def transform_links(link: str):
    name = re.findall(r' applink="/card/([^"]+)"', link)[0]
    return f' applink="/card/{ids[name]}"' if name in ids else ''

rulings = pd.read_csv('Wingspan - Rulings.tsv', sep='\t', header=None, names=['id', 'general', 'specific', 'text', 'source'])
rulings['text'] = rulings.text.map(lambda t: re.sub(r'\\textbf\{([^}]+)\}', r'<strong applink="/card/\1">\1</strong>', re.sub(r'\\textit\{([^}]+)\}', r'<i>\1</i>', re.sub(r"``", r'<i>"', re.sub(r"''", r'"</i>', t)))))
rulings['text'] = rulings.text.map(lambda text: reduce(lambda acc, val: acc.replace(val, transform_links(val)), [x.group() for x in re.finditer( r' applink="/card/([^"]+)"', text)], text))

general = rulings[~pd.isna(rulings['general'])].drop(['specific'], axis=1)
general['general'] = general.general.map(lambda t: re.sub(r'\$\\Rightarrow\$', '➔', t))
specific = rulings[~pd.isna(rulings['specific'])].drop(['general'], axis=1)
specific['specific'] = specific['specific'].map(lambda s: s.strip())
grouped = specific.groupby(by='specific').apply(lambda group: list(map(lambda t: {'text': t[0], 'source': t[1]}, zip(group['text'], group['source']))))

In [7]:
vals =  list(master['Common name'].values) + list(bonus['Name'].values)
list(filter(lambda g: g not in vals, grouped.index))

[]

In [8]:
general_dict = reduce(lambda acc, val: {**acc, val: []}, list(master['Common name']) + list(bonus['Name']), {})
rule_counts = {}

for i, rule in general.iterrows():
    if rule['id'] in general_rulings_map.rulings:
        rule_counter = 0
        for j, row in master.iterrows():
            if general_rulings_map.rulings[rule['id']](row):
                rule_counter += 1
                general_dict[master.loc[j, 'Common name']] += [{'id': rule['id'], 'text': rule['text'], 'source': rule['source']}]
        rule_counts[rule['id']] = rule_counter
    else:
        print(f'Rule {rule["id"]} not yet implemented')

for rules in general_dict.values():
    rules.sort(key=lambda key: rule_counts[key['id']])
    for rule in rules:
        del rule['id']

rule_counts

{'02c': 0,
 '02g': 47,
 '03a': 3,
 '20190122': 0,
 '20190205': 24,
 '20190313': 24,
 '20190601': 76,
 '20190908': 10,
 '20191010': 10,
 '20191202': 0,
 '20191203c': 0,
 '20200109a': 0,
 '20200208': 24,
 '2020022b': 4,
 '20200330': 24,
 '20200404': 319,
 '20200511': 8,
 '20200712': 0,
 '20200716a': 3,
 '20200716b': 22,
 '20201003': 0,
 '20201009': 0,
 '20201116a': 0,
 '20201117': 48,
 '20201211': 0,
 '20210101': 0,
 '20210199a': 0,
 '20210199b': 0,
 '20210206': 4,
 '20210318': 3}

In [9]:
general.reset_index(drop=True, inplace=True)
general.drop(['id'], axis=1, inplace=True)
general.columns = ['name', 'text', 'source']
master['rulings'] = master['Common name'].map(lambda name: grouped[name] if name in grouped else [])
master['additionalRulings'] = master['Common name'].map(lambda name: general_dict[name])
bonus['rulings'] = bonus['Name'].map(lambda name: grouped[name] if name in grouped else [])
bonus['%'] = bonus['%'].map(lambda p: int(p) if type(p) == float else p)

In [10]:
master.sort_values(by='Common name', inplace=True)
bonus.sort_values(by='Name', inplace=True)

In [11]:
master.to_json('generated/master.json', orient='records', index=True)
bonus.to_json('generated/bonus.json', orient='records', index=True)
general.to_json('generated/general.json', orient='index')